# TD - Vectorisation de texte, représentation graphique et classification

Dans ce TD, nous nous intéressons à la représentation vectorielle de texte à partir de la méthode [Bag of words](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Nous étudierons les effets des différents paramètres sur la représentation obtenue. Nous étudierons également deux méthodes pour représenter graphiquement des données vectorielles de grande dimension.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys,os,os.path

%matplotlib notebook

## L'ensemble de textes : SMS Spam Collection

Dans ce TD, nous utiliserons l'ensemble de textes qui se nomme « SMS Spam Collection v1. ». C'est une collection d'environ 5600 SMS, répartis en 2 groupes (spam et ham).

Pour plus de detaille : [site officiel](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/)

**À exécuter -** Pour chargez l'ensemble de textes, utiliez le code suivant

In [ ]:
sms_dataset = pd.read_csv('SMSSpamCollection.txt', sep="\t", header=None).rename(index=str, columns={0: "label", 1: "text"})

**À exécuter -** Vous pouvez afficher un échantillons du dataset de SMS et son analyse avec les codes suivants

In [ ]:
sms_dataset.tail(10)

In [ ]:
sms_dataset.describe()

**Analyse -** L'ensemble de textes est-il équilibré ?

## Bag of words
### Création du dictionnaire

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

**Question -** Créez un `CountVectorizer` et utilisez le corpus de texte pour apprendre le dictionnaire de vectorisation ([CountVectorizer.fit()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit)).

**Question -** Quelle est la taille du dictionnaire obtenu (`CountVectorizer.vocabulary_`) ? Affichez les mots du dictionnaire, que pouvez-vous en conclure ?

### Réduction du nombre de mots dans le dictionnaire

Dans un premier temps, nous utiliserons les méthodes vue en cours pour réduire le nombre de mots dans le dictionnaire.

**À faire -** Àjoutez l'arguement `stop_words='english'` à la contruction du `CountVectorizer` pour retirer les stop words du dictionnaire.

**Question -** Quelle est la taille du dictionnaire alors obtenu ?


Nous avons vu en cours qu'il existe pour chaque langue et pour un grand nombre de mots de cette langue un sous-ensemble de mots dérivés.  Pour réduire la taille du vocabulaire, nous conservons l'élément racine de ces sous ensembles.

Nous allons utiliser la méthode `SnowballStemmer` pour extraire de la racine des mots (stemming).

**À exécuter -** Nous surchargons les classes `CountVectorizer` et `TfidfVectorizer` pour àjouter l'extraction de la racine des mots dans leures méthodes d'analyses.

In [ ]:
import nltk.stem
english_stemmer=nltk.stem.SnowballStemmer('english')
class EnglishStemmedCountVectorizer(CountVectorizer): #EnglishStemmedCountVectorizer hérite de CountVectorizer
    def build_analyzer(self):
        analyzer = super(EnglishStemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
    
class EnglishStemmedTfidfVectorizer(TfidfVectorizer):#EnglishStemmedTfidVectorizer hérite de TfidVectorizer
    def build_analyzer(self):
        analyzer = super(FrenchStemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

**À faire -** Utiliez le vectorizer `EnglishStemmedCountVectorizer`, que nous venons de créer et àjoutez l'arguement `stop_words='english'` à sa contruction pour retirer les stop words du dictionnaire.

**Question -** Quelle est la taille du dictionnaire alors obtenu ?

Maintenant, pour réduire le nombre de mots dans le dictionnaire, nous allons étudier la **fréquence d'apparition des mots dans les documents** ou *document frequency* en anglais.

**Question -** Utilisez la fonction [CountVectorizer.transform()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform) pour vectoriser les documents du corpus de texte. 

**À exécuter -** Pour calculer la fréquence d'apparition de chaque mot dans les documents à partir des documents vectoriser, nous utiliserons la fonction suivante :

In [ ]:
def compute_document_frequency(X):
    df = np.mean(X>0, axis=0)
    df = np.asarray(df).reshape(-1) # convertie un numpy.matrix en numpy.ndarray
    return df

**À exécuter -** Pour affichez l'histogramme de la fréquence d'apparition des mots dans les documents du corpus de texte, nous utiliserons la fonction suivante.

In [ ]:
def plot_hist_df(df, bins=None, yscale='linear'):
    fig = plt.figure(figsize=(6,6));
    ax = fig.gca();
    ax.hist(df,bins);
    ax.set_yscale(yscale);
    ax.set_xbound([np.min(df),np.max(df)]);
    ax.set_xlabel('Document frequency');
    ax.set_ylabel('Nombre de mot');

**Question -** À l'aide des deux fonction que nous venons de définir, affichez l'histogramme de la fréquence d'apparition des mots dans les documents de notre ensemble de SMS

**Analyse -** Qu'est-ce que cela signifie ? Est-il intéressant d'avoir des mots avec une fréquence proche de 1 ou de 0 ?

**Question -** A l'aide de la fonction [CountVectorizer.inverse_transform()](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.inverse_transform), quels sont les mots du dictionnaire qui ont une fréquence trop petite (par exemple avec un df < 0.001) ?

**Question -** A l'aide des parametres **`min_df`** et **`max_df`** de la classe [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), filtrer les mots avec des fréquences indésirables.

Quelle est la taille du dictionnaire alors obtenu ?

Affichez alors le nouvel l'histogramme de la fréquence d'apparition des mots dans les documents du corpus de texte ([matplotlib.pyplot.hist()](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hist)).

## Visualisation

Nous avons vu en cours qu'il est intéressant de visualiser les données pour juger de la qualité de la représentation. 

**À exécuter -**  Pour visualiser en regardant les dimensions deux à deux, nous utilions la fonction suivante

In [ ]:
def plot_croises(X,y,target_names, threshold = 1.0):
    uy = np.unique(y)
    nb_class=len(uy)
    nb_feat=X.shape[1]
    if nb_feat%2:
        ncols=int((nb_feat-1)/2)
        nrows=nb_feat
    else:
        ncols=int(nb_feat/2)
        nrows=nb_feat-1
    plt.figure(figsize=(8, 10))
    col=['b','r','g','c','m','y','k']
    for t in range(nb_class):
        k=1
        for i in range(nb_feat-1):
            for j in range(i+1,nb_feat):
                plt.subplot(nrows,ncols,k)
                
                x_t = X[y == uy[t],i]
                y_t = X[y == uy[t],j]
                
                x_mean = np.mean(x_t)
                y_mean = np.mean(y_t)
                
                dist = (x_t-x_mean)**2 + (y_t-y_mean)**2
                
                
                dist_max = np.sort(dist)[int(np.floor((dist.shape[0]-1)*threshold))]
                
                plt.scatter(x_t[dist<dist_max], y_t[dist<dist_max], color=col[t],lw=2,marker='o',label=target_names[t], alpha=0.5)
                plt.ylabel('Dim. ' + str(j+1))
                plt.xlabel('Dim. ' + str(i+1))
                k=k+1
    plt.legend(bbox_to_anchor=(2,2))  
    plt.tight_layout()  
    

**À exécuter -** Nous pouvons alors visualiser les 6 premières dimensions de notre représentation par Bag of words

In [ ]:
plot_croises(X[:,:6],sms_dataset['label'],sms_dataset['label'].unique())

**Analyse -** Que constatez vous ?

### Analyse en Composantes Principales

Une méthodes tres couramments utilisée qui permet de visualiser les données est l'utilisation de [Analyse en Composantes Principales](https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales)(ACP), qui est disponible dans [sklearn.decomposition.PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

In [ ]:
from sklearn.decomposition import PCA

**Question** A l'aide de l'ACP, réduisez la dimension de la signature à 6 dimensions et visualisez-les deux à deux avec la fonction `plot_croises()`.

**Analyse -** Le résultat obtenu est-il plus facilement lisible ? Pourquoi ? 

## Discrimination par régression logistique

### Mise en oeuvre

Nous aurons besoin de la classe `LogisticRegression`, d'une méthode découpant en un ensemble d'apprentissage et de test, `train_test_split` et éventuellement des méthodes d'analyse et rapport `confusion_matrix`, `classification_report`.

**À exécuter -** Celles-ci sont importées par

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report    

**À faire -**
 - définir une base de test et une base d'apprentissage en utilisant la méthode `train_test_split`
 - instancier la classe `LogisticRegression`, par exemple sous le nom `cls`, puis apprendre par la méthode `fit` et prédire par `predict`. 

**À faire -** Visualiez quels sont les coefficients obtenus par `cls.intercept_` et `cls.coef_`

**À faire -** Utiliez la fonction `classification_report` pour visualiez les performances de classification

**À exécuter -** Pour afficher la matrice de confusion, nous utiliserons la fonction suivante

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

**À faire -** Utiliez la fonction `confusion_matrix` pour calculer la matrice de confusion et utiliez la fonction que nous venons de définir pour l'afficher

**Analyse -** Analysez la matrice de confusion obtenue, le résultat est-il cohérent avec la visualisation obtenue en utilisant l'ACP ?

**À faire -** Àjoutez l'arguement `class_weight='balanced'` à la contruction de `LogisticRegression`.

**Question -** Observez les effets sur les performances de classification

**À faire -** Faites varier le parametre **`min_df`** de la classe [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

**Question -** Observez les effets sur la visualisation et sur les performances de classification